# RL Metrics Dashboard (Plotly)

Этот ноутбук визуализирует метрики обучения из файлов `rl_training_metrics.parquet` и/или `rl_training_metrics.csv`.

- Обновляйте метрики, запуская обучение.
- Используйте функции загрузки и построения графиков ниже.
- Графики интерактивные (Plotly): масштабирование, панорама, выбор диапазона.



In [ ]:
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from typing import Optional, Tuple

PARQUET_PATH = Path("rl_training_metrics.parquet")
CSV_PATH = Path("rl_training_metrics.csv")


def load_metrics(parquet_path: Path = PARQUET_PATH, csv_path: Path = CSV_PATH) -> pl.DataFrame:
    """Загрузить метрики из parquet или csv. Предпочтение parquet.
    Возвращает Polars DataFrame с приведёнными типами.
    """
    if parquet_path.exists():
        df = pl.read_parquet(parquet_path)
    elif csv_path.exists():
        df = pl.read_csv(csv_path)
    else:
        raise FileNotFoundError("Не найдено ни rl_training_metrics.parquet, ни rl_training_metrics.csv")

    # Приводим типы и ожидаемые колонки
    expected = [
        "episode",
        "objective",
        "epsilon",
        "total_assigned",
        "active_couriers",
        "avg_courier_time",
        "max_courier_time",
        "remaining_polygons",
    ]
    for col in expected:
        if col not in df.columns:
            df = df.with_columns(pl.lit(None).alias(col))

    df = df.select(expected)
    return df.sort("episode")


def fig_objective(df: pl.DataFrame) -> go.Figure:
    fig = px.line(df.to_pandas(), x="episode", y="objective", title="Objective per episode")
    fig.update_layout(yaxis_title="objective")
    return fig


def fig_epsilon(df: pl.DataFrame) -> go.Figure:
    fig = px.line(df.to_pandas(), x="episode", y="epsilon", title="Epsilon schedule")
    return fig


def fig_assignment(df: pl.DataFrame) -> go.Figure:
    fig = px.line(df.to_pandas(), x="episode", y=["total_assigned", "remaining_polygons"], title="Assigned vs Remaining")
    fig.update_layout(yaxis_title="count")
    return fig


def fig_courier_times(df: pl.DataFrame) -> go.Figure:
    fig = px.line(df.to_pandas(), x="episode", y=["avg_courier_time", "max_courier_time"], title="Courier times (avg/max)")
    fig.update_layout(yaxis_title="seconds")
    return fig


def build_dashboard(df: pl.DataFrame) -> None:
    display(fig_objective(df))
    display(fig_epsilon(df))
    display(fig_assignment(df))
    display(fig_courier_times(df))


# Автозагрузка при первом запуске
try:
    _df = load_metrics()
    build_dashboard(_df)
except FileNotFoundError as e:
    print(str(e))



In [ ]:
# Утилита обновления (перезагрузка из файлов)

def refresh_and_render():
    df = load_metrics()
    build_dashboard(df)

# Пример запуска:
# refresh_and_render()

